<a href="https://colab.research.google.com/github/dyjdlopez/fund-of-aiml/blob/main/activities/05%20-%20Classification/fund_aiml_05v1_lec2_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic 05.2: Non-Linear Classification
$_{\text{©D.J. Lopez | 2021 | Fundamentals of Machine Learning}}$



### Part 1: Perceptron Algorithm
The Perceptron was first conceptualized by Frank Rosenblatt in his paper [The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.335.3398&rep=rep1&type=pdf) in 1958. The perceptron is one of the earliest mathematical conceptualization of a brain neuron. In simplest terms, a perceptron does a weighted sum of all inputs and then performs an activation. In the early implementations of the perceptron the activation used was a step function described as:
$$step(z) = \left\{
  \begin{array}\\
    1 \text{ if } \ b+ \sum w_iX_n\geq 0 \\
    0 \text{ otherwise}
    \end{array}
\right.
$$
![image](https://jontysinai.github.io/assets/article_images/2017-11-11-the-perceptron/bio-vs-MCP.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:

N = 100
cov = 0
center1 = np.array([0, 1.7]).T
cov_mat1 = np.array([
                    [1,cov],
                    [cov,1]
])

center2 = np.array([0, -1.7]).T
cov_mat2 = np.array([
                    [1,cov],
                    [cov,1]
])

np.random.seed(0)
X1 = np.array(np.random.multivariate_normal(center1,cov_mat1,int(N/2)))
X2 = np.array(np.random.multivariate_normal(center2,cov_mat2,int(N/2)))
## We'll declare another gaussian distribution with a different mean

plt.figure(figsize=(8,8))
plt.scatter(X1[:,0],X1[:,1], c='r', label='1')
plt.scatter(X2[:,0],X2[:,1], c='b', label='0')
plt.ylabel("Some Target")
plt.xlabel("Some Feature")
plt.legend()
plt.grid()
plt.show()

In [ ]:
def step_activation(z):
  """
  Compute the step activation of z

  Arguments:
  z -- A scalar or numpy array of any size.

  Return:
  filtered step activations step(z)
  """
  return np.where(z>=0, 1,0)

In [ ]:
def init_weights(dim):
  """
  Does a zero-initialization of the weights and bias

  Arguments:
  dim -- Desired dimension for the weights.

  Return:
  w -- initialized weights
  b -- initilaized bias
  """
  w = np.zeros(shape=(dim,1))
  b = 0
  return w, b

In [ ]:
def sum_err(preds,y):
  """
  Computes the Sum of Squared Errors for a set of predictions
  and truth values

  Arguments:
  preds -- Set of predictions.
  y -- Set of truth values

  Return:
  sse -- Sum of the squared errors
  """
  sse = np.sum(np.square(y-preds))
  return sse

In [ ]:
def accuracy(preds, Y):
  """
  Computes the accuracy for a set of predictions
  and truth values

  Arguments:
  preds -- Set of predictions.
  y -- Set of truth values

  Return:
  accuracy -- Computed accuracy
  """
  accuracy = 1-np.mean(np.abs(preds-Y))
  return accuracy

In [ ]:
def propagate(X,y,w,b):

  # Compute for the transformed vector of the 
  # dataset w.r.t the weights and biases
  z = (X@w) + b

  # Compute for the step activation
  A = step_activation(z)

  # Compute for the prediction error
  error = A-y
  acc = accuracy(y,A)

  # Update the weights and biases
  # Learning/Update routine
  w = np.dot(X.T,error)
  b = np.sum(error)

  # Compute the cost
  cost = sum_err(A,y)

  # Store the parameters in a dictionary for tracking
  grads = {"dw": w,
           "db": b}
  
  return grads, cost, acc

In [ ]:
w,b = init_weights(X_train.shape[1])
propagate(X_train,y_train,w,b)

In [ ]:
def train(w, b, X, y, lr, epochs, early_stopping=True, stop_thresh=0.9):
  costs = []
  accuracies = []

  for i in range(epochs):
    # Do a forward propagation to obtain the gradients
    grads, cost, accuracy = propagate(X,y,w,b)

    # Locally store the gradients    
    dw=grads['dw']
    db=grads['db']

    # Update routine per epoch
    w = w - lr*dw
    b = b - lr*db

    # Store the costs per epoch for logs      
    
    if i % 20 == 0:
      print (f"Epoch {i}: Loss: {cost} Accuracy: {accuracy}")
      costs.append(cost)
      accuracies.append(accuracy)

    # Store the learned parameters for logs
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    if early_stopping and accuracy >= stop_thresh:
      print(f"Target metric met, stopping the training at {i} epoch(s).\n")
      break

  return params, grads, costs

In [ ]:
w,b = init_weights(X_train.shape[1])
learning_rate = 1
epochs = 100

params, grads, costs = train(w, b, X_train, y_train, 
                             lr=learning_rate, epochs=epochs, 
                             early_stopping=True, stop_thresh=1.0)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

In [ ]:
def predict(X, weights, bias):
  z = (X@weights)+bias
  return np.where(z>=0, 1,0)


In [ ]:
weights = params["w"]
bias = params["b"]
preds = predict(X_test,weights,bias)
accuracy(y_test, preds)

In [ ]:


c_matrix = confusion_matrix(y_test, preds)
sns.heatmap(c_matrix, annot=True)
plt.xlabel("Ground Truths")
plt.ylabel("Predicted")
plt.show()

## Part 2: Back-propagation
Although the Perceptron with the step activation produces good results producing a linear classifier, it lacks another fundamental technique for being a robust neural network model—Backpropagation. Backpropagation is a short form for "backward propagation of errors." It is a method of training artificial neural networks. This method helps to calculate the gradient of a loss function for all the weights in the network. <br>
In this section, we will use a sigmoid function as an activation function instead of a step activation. Since backpropagation will not be effective with the step function its gradient (derivative) is zero, and that will not be useful for computing the loss function.<br>
<b>Loss Function</b><br>
A loss function is the function we want to minimize or maximize is called the objective function or criterion. When we are minimizing it, we may also call it the cost function, loss function, or error function [[1]](https://www.deeplearningbook.org/contents/numerical.html). <br>
To save you the time and brainpower, for our example our loss function is:
$$J(\theta)=\frac{1}{m} \sum^m_{i}cost(h_{\theta}(x^{(i)}, y^{(i)}) \\
\text{if y = 1} : -\log{(h_\theta(x))}\\
\text{if y = 0} : -\log{(1-h_\theta(x))}\\
J(\theta)=-\frac{1}{m} \sum^m_{i}{y^{(i)}\log{(h_\theta(x))}+(1-y^{(i)})(\log{(1-h_\theta(x))}} \\
J(\theta)=-\frac{1}{m} \sum^m_{i}{Y^T\log(h)+(1-Y)^T\log(1-h)}
$$

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1 / (1 + np.exp(-z))    
    return s

In [ ]:
def transfer_derivative(d):
  return d*(1.0-d)

In [ ]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array 
    b -- bias, a scalar
    X -- data of size
    Y -- true "label" vector 

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    """
    
    m = X.shape[0]
    alpha = 10**-8
    
    # FORWARD PROPAGATION (FROM X TO COST)
    h = sigmoid((X@w)+b)                                   # compute activation
    J = -1 / m * np.sum(Y * np.log(h+alpha) + (1-Y) * np.log((1-h)+alpha))  # compute cost
    error = (h-Y)*transfer_derivative(h)
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = 1/m * X.T @ error
    db = 1/m * np.sum(error)

    cost = np.squeeze(J)
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
def optimize(w, b, X, Y, epochs, lr, print_cost = True):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size
    b -- bias, a scalar
    X -- data of shape 
    Y -- true "label" vector
    epochs -- number of iterations of the optimization loop
    lr -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(epochs):
        
        
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - lr * dw
        b = b - lr * db
        
        # Record the costs
        if i % 50 == 0:
            costs.append(cost)
        
        # Print the cost every 10 training iterations
        if print_cost and i % 50 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
w,b = init_weights(X_train.shape[1])
learning_rate = 0.1
epochs = 100
params, grads, costs = optimize(w, b, X_train, y_train, 
                             lr=learning_rate, epochs=epochs)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

In [ ]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size
    b -- bias, a scalar
    X -- data of size 
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    A = sigmoid((X@w)+b) 
    Y_prediction = np.where(A>=0.5,1,0)
    
    return Y_prediction

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 10, learning_rate = 0.5, print_cost = True):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    # initialize parameters with zeros
    w, b = init_weights(X_train.shape[1])

    # Gradient descent 
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
neuron_model = model(X_train, y_train, X_test, y_test, num_iterations=100, learning_rate=1)

In [ ]:
c_matrix = confusion_matrix(y_test, neuron_model['Y_prediction_test'])
sns.heatmap(c_matrix, annot=True)
plt.xlabel("Ground Truths")
plt.ylabel("Predicted")
plt.show()

In [ ]:
print(f"F1 Score: \t{f1_score(y_test, neuron_model['Y_prediction_test'])}")
print(f"Recall: \t{recall_score(y_test, neuron_model['Y_prediction_test'])}")
print(f"Precision: \t{precision_score(y_test, neuron_model['Y_prediction_test'])}")

In [ ]:
plot_weights(X_train,neuron_model['w'],neuron_model['b'])

## Laboratory Activity
1. For the laboratory activity, obtain a dataset of your liking from a data source. Explain the purpose of the dataset and mention any publication if it is obtained from the source. Provide a needs statement and significance for the dataset.

2. Identify an algorithm or method in performing a single or multiple variable classification using the Perceptron alogrithm. 

3. You must re-create your Perceptron algorithm with Gradient Descent and Backpropagation using your own code in a separate Google Colab. However, you are required to observe the following:

>* Enforce object-oriented programming by implementing at least two of the pillars of OOP in the entirety of the solution.
* Dedicated functions for training, predicting, and evaluating the solution.
* A DataFrame of the metrics of the solution
* A visualization of the solution’s results.
